# INSTALL & IMPORTS

In [1]:
#pip install selenium -q
#pip install requests -q
# pip install bs4 -q 

In [1]:
import requests #  to perform web requests in Python due to its simplicity and power.
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# TESTING

In [42]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [81]:
page_source = driver.page_source
profile = BeautifulSoup(page_source, 'html.parser')
name = profile.find("span", class_='encounters-story-profile__name').text
age = profile.find("span", class_= "encounters-story-profile__age").text
about = profile.find("ul", class_= "encounters-story-about__badges").text if profile.find("ul", class_= "encounters-story-about__badges") else ""
if about:
    about = re.split('(?=[A-Z])', about)

location = profile.find("span", class_="header-2 text-color-black").text
more_info = profile.find("p", class_="encounters-story-about__text").text
job = profile.find("p", class_="encounters-story-profile__occupation").text
education = profile.find("p", class_="encounters-story-profile__education").text


print(name, age, about, location, more_info, job, education)

Joan , 30 ['174 cm', 'Active', 'Aries', 'Graduate degree', 'Socially', 'Never', 'Man', 'Something casual', 'Open to kids', 'Agnostic'] Barcelona Buscant còmplice per tast de cerveses

That's what she said Advocat Universitat Pompeu Fabra 2015


In [83]:
education

'Universitat Pompeu Fabra 2015'

In [54]:
# Esperar a que el botón esté presente y sea clickeable
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
    )
    button.click()
    print("Botón clickeado con éxito")
except Exception as e:
    print("No se pudo clickear el botón:", e)


Botón clickeado con éxito


# FUNCTION TO TAKE ALL INFO

In [2]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [26]:
def extract_profile_info():

    profiles = []
    prevJob = ""
    prevStories = ""
    prevEducation = ""
    
    for i in range(121):
        
        # Obtener el contenido de la página actual
        page_source = driver.page_source
    
        # Analizar el contenido de la página con BeautifulSoup
        profile = BeautifulSoup(page_source, 'html.parser')
    
    
        # Realizar un diccionario para que la información se vaya guardando.
        profiles_dict = {}
    
        # Esperar 15 segundos antes de extraer la información
        time.sleep(5)
    
        # Extraer información del perfil
        name = profile.find("span", class_='encounters-story-profile__name')
        profiles_dict["name"] = name.text if name else None
        
        age = profile.find("span", class_= "encounters-story-profile__age")
        profiles_dict["age"] = age.text.strip(",")  if age else None

        
        location = profile.find("span", class_="header-2 text-color-black")
        profiles_dict["location"] = location.text if location else ""
    

        story_about = profile.find("p", class_="encounters-story-about__text")
        profiles_dict["story_about"] = story_about.text if story_about and story_about.text != prevStories else None
        if story_about:
            prevStories = story_about.text

        
        job = profile.find("p", class_="encounters-story-profile__occupation")
        profiles_dict["job"] = job.text if job and job.text != prevJob else None
        if job:
            prevJob = job.text
    
        education = profile.find("p", class_="encounters-story-profile__education")
        profiles_dict["education"] = education.text if education and education.text != prevEducation else None
        if education:
            prevEducation = education.text

        # Encuentra todos los elementos div con la clase "pill"
        about = profile.find_all("div", class_="pill")  

        #categories_dict = {}
        
        for pill in about: # Pill como elemento de la lista
            image_tag = pill.find("img")
            
            if image_tag is not None: # Verificación de la existencia de una etiqueta de imagen
                image_url = image_tag['src'] # Extracción de la URL de la imagen
                text = pill.get_text(strip=True) # Extracción de texto del elemento pill 
                # strip=True se utiliza para eliminar espacios en blanco alrededor del texto
                category = image_url.split('_')[-1].split('.')[0] # Extracción de la categoría de la imagen
                if category is not None and category not in profiles_dict: 
                    # Verificación de la existencia de la categoría y actualización del diccionario
                    # Asignación de valores None cuando no se encuentra una etiqueta de imagen
                    profiles_dict[category] = text # Add to profiles
            
            else:
                category = None
                text = None
                
    

        profiles.append(profiles_dict)
    
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
            )
            time.sleep(5)  # Añadir un temporizador adicional de 5 segundos
            button.click()
            print("Botón clickeado con éxito")
        except Exception as e:
            print("No se pudo clickear el botón:", e)
            

    return profiles

In [27]:
profile_info = extract_profile_info() 

# Make a DataFrame from a dict. 
df = pd.DataFrame(profile_info)

# Print DataFrame
df

Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clicke

,name,age,location,story_about,job,education,exercisev2,drinkingv2,smokingv2,genderv2,starSignv2,religionv2,intentionsv2,heightv2,familyPlansv2,educationv2,Politicsv2
0,Viktor,26,Barcelona,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jagoda,25,Barcelona,None,None,None,Sometimes,Socially,Regularly,Woman,Aquarius,Agnostic,NaN,NaN,NaN,NaN,NaN
2,Rongge,25,Barcelona,Not top…,None,None,Sometimes,Socially,Regularly,Man,Aquarius,Agnostic,Something casual,NaN,NaN,NaN,NaN
3,Julia,25,Barcelona,None,None,None,Active,Frequently,Socially,Woman,Pisces,NaN,Don’t know yet,173 cm,Want someday,NaN,NaN
4,Taro,30,Barcelona,"Japan-based, looking to visit Europe layer thi...",Project Manager,Applied Linguistics Major,Active,Socially,Never,Man,Gemini,Agnostic,Relationship,169 cm,Open to kids,Undergraduate degree,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Émilie,23,Barcelona,"Hey, love coffee and beer so let’s grab some t...",None,None,NaN,NaN,NaN,Woman,Virgo,NaN,NaN,172 cm,NaN,NaN,NaN
117,Luana,32,Barcelona,"@luanadelazzari\n\nFrom the best country 🇧🇷, l...",None,None,Sometimes,Socially,Never,Woman,Taurus,NaN,NaN,174 cm,NaN,NaN,NaN
118,Nerea,22,Barcelona,None,None,None,Active,Socially,Never,Woman,Scorpio,NaN,Don’t know yet,174 cm,NaN,NaN,NaN
119,Giuliana,28,Barcelona,🇧🇷,None,None,Active,Socially,Never,Woman,Capricorn,NaN,Don’t know yet,NaN,NaN,In grad school,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           121 non-null    object
 1   age            121 non-null    object
 2   location       121 non-null    object
 3   story_about    80 non-null     object
 4   job            33 non-null     object
 5   education      30 non-null     object
 6   exercisev2     108 non-null    object
 7   drinkingv2     112 non-null    object
 8   smokingv2      109 non-null    object
 9   genderv2       115 non-null    object
 10  starSignv2     112 non-null    object
 11  religionv2     74 non-null     object
 12  intentionsv2   116 non-null    object
 13  heightv2       114 non-null    object
 14  familyPlansv2  83 non-null     object
 15  educationv2    103 non-null    object
 16  Politicsv2     49 non-null     object
dtypes: object(17)
memory usage: 16.2+ KB


In [29]:
df.to_csv('man8.csv', index=False)

In [30]:
df["genderv2"].value_counts()

genderv2
Woman          67
Man            45
Cis man         2
Trans woman     1
Name: count, dtype: int64